# Install packages
Install tiktoken and openai packages

In [ ]:
# install necessary packages
pip install tiktoken
pip install openai

Get ada v2 embeddings via openAI API

In [ ]:
import openai

openai.api_key = "inser your api key here :)"

embedding = openai.Embedding.create(
    input="Your text goes here", model="text-embedding-ada-002",
)["data"][0]["embedding"]
len(embedding)

1536

In [ ]:
response = openai.Embedding.create(
    input="Your text string goes here",
    model="text-embedding-ada-002"
)
embeddings = response['data'][0]['embedding']
print(embeddings)

[-0.006968616973608732, -0.005271822679787874, 0.01190108060836792, -0.024984877556562424, -0.024554798379540443, 0.03967474773526192, -0.010140446946024895, -0.009481888264417648, -0.013245075941085815, -0.01000604685395956, -0.011511322110891342, 0.007775014266371727, -0.014138833619654179, 0.007741414476186037, 0.010147166438400745, -0.005013103596866131, 0.02294200472533703, -0.001628754660487175, 0.014985550194978714, -0.010281565599143505, 0.00485854409635067, 0.01247227843850851, 0.004835024010390043, 0.010819164104759693, -0.00661581801250577, -0.0003509088419377804, 0.0055708615109324455, -0.012613398022949696, 0.016369866207242012, 0.0044587054289877415, 0.006639338098466396, -0.007116456516087055, -0.015160270035266876, -0.00662589818239212, -0.018533699214458466, 0.004062226507812738, 0.003178549464792013, -0.018963776528835297, 0.030320540070533752, -0.007526375353336334, 0.008117733523249626, 0.009481888264417648, -0.001102916314266622, -0.0004422585479915142, -0.00877629

Next, we import the csv file of our Quran QA-Pairs

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving islamqa_10thousand_filtered.csv to islamqa_10thousand_filtered.csv


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('islamqa_10thousand_filtered.csv', lineterminator="\n", encoding ="utf-8-sig")

Now we load the ada02 embedding model. Since this is the 2nd generation of ada embeddings, we hope to achieve good performance with these.

In [ ]:
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

Now we drop the URL axis since we do not need it for now

In [ ]:
df = df.drop(["URL"], axis=1)
# df.head()
df.columns = [x.replace("\r", "") for x in df.columns.to_list()]
df

,Title,Question,Answer
0,Interruption of Wudhu,"If during wudhu (ablution), one finds dirt stu...",Praise be to Allah. Continuity would not be di...
1,Wudu while wounded,What should a person do if one of the areas no...,Praise be to Allah.If a person is wounded in o...
2,Time of occurence of janaba unknown.,If a man sees the signs of sexual discharge im...,Praise be to Allah.If a man sees the signs of ...
3,Is Circumcision Necessary for Conversion?,A lawyer in Argentina is asking about the ruli...,Contents Related Male circumcision in Islam Is...
4,When Can You Pray After a Miscarriage?,If a woman has a miscarriage and has discharge...,Related Praise be to Allah.This case depends o...
...,...,...,...
1538,Ruling on reading Qur’aan over the grave of a ...,Some people in our village bring together some...,Praise be to Allah.This action is bid’ah (repr...
1539,His mother swore that she would never give him...,my mother swore to me she would not gove me an...,Praise be to Allah.There is nothing wrong with...
1540,Ruling on grave markers on which are written d...,Is it permissible to put a piece of metal or a...,Praise be to Allah.It is not permissible to wr...
1541,Male nurse being alone with female nurse in th...,I am a male nurse and my work is nursing men. ...,Praise be to Allah.It is not permissible for t...


With the tiktoken module, we encode the embeddings

In [ ]:
import tiktoken
encoding = tiktoken.get_encoding(embedding_encoding)

In [ ]:
df.head()

,Title,Question,Answer
0,Interruption of Wudhu,"If during wudhu (ablution), one finds dirt stu...",Praise be to Allah. Continuity would not be di...
1,Wudu while wounded,What should a person do if one of the areas no...,Praise be to Allah.If a person is wounded in o...
2,Time of occurence of janaba unknown.,If a man sees the signs of sexual discharge im...,Praise be to Allah.If a man sees the signs of ...
3,Is Circumcision Necessary for Conversion?,A lawyer in Argentina is asking about the ruli...,Contents Related Male circumcision in Islam Is...
4,When Can You Pray After a Miscarriage?,If a woman has a miscarriage and has discharge...,Related Praise be to Allah.This case depends o...


Now we combine QA-Pairs into one single text.

In [ ]:
df["combined"] = (
    "Title: " + df.Title.str.strip() + "; Question: " + df.Question.str.strip() + "; Answer: " + df.Answer.str.strip()
)

In [ ]:
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df

,Title,Question,Answer,combined,n_tokens
0,Interruption of Wudhu,"If during wudhu (ablution), one finds dirt stu...",Praise be to Allah. Continuity would not be di...,Title: Interruption of Wudhu; Question: If dur...,207
1,Wudu while wounded,What should a person do if one of the areas no...,Praise be to Allah.If a person is wounded in o...,Title: Wudu while wounded; Question: What shou...,137
2,Time of occurence of janaba unknown.,If a man sees the signs of sexual discharge im...,Praise be to Allah.If a man sees the signs of ...,Title: Time of occurence of janaba unknown.; Q...,492
3,Is Circumcision Necessary for Conversion?,A lawyer in Argentina is asking about the ruli...,Contents Related Male circumcision in Islam Is...,Title: Is Circumcision Necessary for Conversio...,468
4,When Can You Pray After a Miscarriage?,If a woman has a miscarriage and has discharge...,Related Praise be to Allah.This case depends o...,Title: When Can You Pray After a Miscarriage?;...,607
...,...,...,...,...,...
1538,Ruling on reading Qur’aan over the grave of a ...,Some people in our village bring together some...,Praise be to Allah.This action is bid’ah (repr...,Title: Ruling on reading Qur’aan over the grav...,990
1539,His mother swore that she would never give him...,my mother swore to me she would not gove me an...,Praise be to Allah.There is nothing wrong with...,Title: His mother swore that she would never g...,542
1540,Ruling on grave markers on which are written d...,Is it permissible to put a piece of metal or a...,Praise be to Allah.It is not permissible to wr...,Title: Ruling on grave markers on which are wr...,207
1541,Male nurse being alone with female nurse in th...,I am a male nurse and my work is nursing men. ...,Praise be to Allah.It is not permissible for t...,Title: Male nurse being alone with female nurs...,445


Since ada02 only accepts texts with a token length below 8191, we have to discard all QA-Pair texts which exceed this limit.

In [ ]:
# check for exceeded rows
exceeded = df[(df.n_tokens > 8191)]

In [ ]:
max_tokens = 8191
df = df[df.n_tokens <= max_tokens]

Now we finally get the embeddings for each QA-Pair text by applying the embedding model on the *combined* column.

In [ ]:
from openai.embeddings_utils import get_embedding

df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))

In [ ]:
df.to_csv("islamqa_10thousand_filtered_with_embeddings.csv", encoding="utf-8-sig")